In [1]:
# change current folder to working directory
import os

os.chdir("/home/nlp/achimoa/workspace/ModernHebrewBERT")

In [3]:
from typing import cast
from omegaconf import OmegaConf as om, DictConfig
from datetime import datetime

yaml_path = "yamls/main/base/flex-bert-rope-phase-0.1-pretrain.yaml"
with open(yaml_path) as f:
    yaml_cfg = om.load(f)
cfg = cast(DictConfig, yaml_cfg)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
cfg.timestamp = timestamp

resolved_config = om.to_container(cfg, resolve=True)
cfg = om.create(resolved_config)

cfg["init_from_checkpoint"] = {
    "checkpoint_cfg": cfg.model,
    "checkpoint_load_path": "checkpoints",
    "checkpoint_run_name": "modern-bert-base-phase-0.1-pretrain/20250521_2334/ckpt"
}

In [5]:
from pathlib import Path
from main import init_from_checkpoint, build_model

# 1. Get HuggingFace model from Composer
flex_bert_model = build_model(cfg.model)
init_from_checkpoint(cfg.init_from_checkpoint, flex_bert_model)

Initializing model from checkpoint modern-bert-base-phase-0.1-pretrain/20250512_1506/ckpt/latest-rank0.pt


/home/nlp/achimoa/workspace/ModernHebrewBERT/main.py:385: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(_ensure_valid_checkpoint(checkpoint_filepath), map

Initalized model from checkpoint modern-bert-base-phase-0.1-pretrain/20250512_1506/ckpt with n_params=1.6838e+08 parameters


In [ ]:
import torch
from transformers import AutoTokenizer

# Switch to eval mode
model = flex_bert_model.model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Load tokenizer and encode
tokenizer = AutoTokenizer.from_pretrained('/home/nlp/achimoa/workspace/ModernHebrewBERT/models/modern-bert-base-phase-0.1-pretrain/20250521_1814/ckpt')
text = "Your sentence here."
encoded = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
input_ids = encoded['input_ids'].to(device)
attention_mask = encoded['attention_mask'].to(device)

# Inference
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    embedding = outputs[0][:, 0, :]  # [CLS] pooling
    # or embedding = outputs[0].mean(dim=1)  # mean pooling

print(embedding.shape)


/home/nlp/achimoa/miniconda3/envs/bert24/lib/python3.11/site-packages/torch/_inductor/compile_fx.py:150: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


IndexError: too many indices for tensor of dimension 2

In [14]:
# print(outputs)
print(outputs.__dict__)
# print(outputs.keys())  # Try this; sometimes the output acts like a dict

{'loss': None, 'logits': tensor([[-2.2580e-01,  1.6077e-01, -5.3962e+00,  ..., -6.7045e-01,
         -8.9971e-01, -5.9641e-02],
        [-3.1319e-01,  5.1377e-01, -6.0622e+00,  ..., -8.3176e-01,
         -3.1419e-01, -3.5176e-01],
        [-5.8954e-01,  1.5424e+00, -6.1247e+00,  ..., -8.7751e-01,
          2.6475e-01, -2.7393e-02],
        [ 1.2775e-01,  4.5350e-01, -6.0917e+00,  ..., -1.1803e+00,
          1.9247e-01, -2.6176e-01],
        [ 2.1294e-01,  1.2724e+00, -6.2624e+00,  ..., -1.1219e+00,
          4.8537e-01, -9.0592e-01],
        [-1.7215e-01,  1.7053e+00, -6.1493e+00,  ..., -6.5197e-01,
          2.6843e-03, -6.0781e-01]], device='cuda:0'), 'hidden_states': None, 'attentions': None, 'indices': tensor([0, 1, 2, 3, 4, 5], device='cuda:0'), 'cu_seqlens': tensor([0, 6], device='cuda:0', dtype=torch.int32), 'max_seqlen': 6, 'batch_size': 1, 'seq_len': 6, 'labels': None}


In [12]:
outputs.hidden_states

In [6]:
from transformers import ModernBertModel

model_path = "/home/nlp/achimoa/workspace/ModernHebrewBERT/models/modern-bert-base-phase-0.1-pretrain/20250521_1200/ckpt"
model = ModernBertModel.from_pretrained(model_path)

You are using a model of type bert to instantiate a model of type modernbert. This is not supported for all configurations of models and can yield errors.
Some weights of ModernBertModel were not initialized from the model checkpoint at /home/nlp/achimoa/workspace/ModernHebrewBERT/models/modern-bert-base-phase-0.1-pretrain/20250521_1200/ckpt and are newly initialized: ['embeddings.norm.weight', 'embeddings.tok_embeddings.weight', 'final_norm.weight', 'layers.0.attn.Wo.weight', 'layers.0.attn.Wqkv.weight', 'layers.0.mlp.Wi.weight', 'layers.0.mlp.Wo.weight', 'layers.0.mlp_norm.weight', 'layers.1.attn.Wo.weight', 'layers.1.attn.Wqkv.weight', 'layers.1.attn_norm.weight', 'layers.1.mlp.Wi.weight', 'layers.1.mlp.Wo.weight', 'layers.1.mlp_norm.weight', 'layers.10.attn.Wo.weight', 'layers.10.attn.Wqkv.weight', 'layers.10.attn_norm.weight', 'layers.10.mlp.Wi.weight', 'layers.10.mlp.Wo.weight', 'layers.10.mlp_norm.weight', 'layers.11.attn.Wo.weight', 'layers.11.attn.Wqkv.weight', 'layers.11.attn

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from pathlib import Path

# model_path = "/home/nlp/achimoa/workspace/ModernHebrewBERT/models/modern-bert-base-phase-0.1-pretrain/20250521_2334/ckpt"
model_path = "/home/nlp/achimoa/workspace/ModernBERT/hf_exported_model/ModernBERT-Hebrew-base_20250522_0255"
# model_path = "answerdotai/ModernBERT-base"
model = AutoModelForMaskedLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [4]:
tokenizer.vocab

{'גיע': 785,
 'ured': 4964,
 'nOb': 59703,
 '▁learners': 53588,
 'בינק': 68298,
 '▁עוונות': 85009,
 '▁Adapt': 96532,
 '▁במטרו': 67110,
 '▁המ': 40,
 '▁professionals': 16322,
 'לוזורוב': 54109,
 '▁repertoire': 98841,
 '▁Physical': 59631,
 '▁עבירות': 11035,
 '▁קומוניסטים': 88328,
 '▁מלג': 58363,
 '▁לרכב': 11270,
 'AN': 3894,
 'דברים': 17817,
 'טכנולוגיה': 42028,
 '▁exterior': 44504,
 '▁הרפור': 35605,
 'cakes': 68373,
 '▁האפרי': 90425,
 'nSecond': 49138,
 '▁צהל': 47049,
 '▁שלזוויג': 87088,
 '▁לעין': 18708,
 '▁מונרו': 51250,
 'מתק': 84779,
 '▁המרץ': 85952,
 'trunc': 95027,
 'באום': 62238,
 'פגין': 42626,
 '▁החוזק': 86415,
 '▁ועומדים': 74703,
 'ishi': 74311,
 '▁1994,': 40480,
 '▁wheels': 34989,
 '▁בסנטה': 86258,
 '▁ועברה': 40918,
 '▁נגזרו': 71184,
 'zh': 23944,
 '▁backing': 48119,
 '▁נוטלת': 96561,
 '▁סנטיאגו': 73035,
 '▁פרישה': 26018,
 '▁סקריפט': 69751,
 '▁קוצ': 27784,
 '▁שהכינו': 89697,
 '▁התפרסמו': 39033,
 '▁מנת': 1657,
 '▁שהפעילו': 93588,
 '▁לכידת': 66299,
 '96': 5552,
 '–': 99965,
 '▁20

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tokenizer", use_fast=False)

OSError: tokenizer is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`